In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome('../driver/chromedriver')
driver.get("https://open.neis.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=OPEN17320190722180924242823&infSeq=1")

office_list_raw = driver.find_element_by_xpath("""//*[@id="sheet-filter-ATPT_OFCDC_SC_CODE"]""")
office_list = office_list_raw.find_elements_by_tag_name("option") 

office_names = [option.text for option in office_list] # 시도교육청코드 문자열 가져오기
del office_names[0] #'(필수)선택하세요' 항목 삭제

import time
from tqdm import tqdm_notebook
from selenium.webdriver.common.alert import Alert

alert = Alert(driver)

school_names = driver.find_element_by_xpath("""//*[@id="sheet-filter-SCHUL_NM"]""")
school_names.send_keys("고등학교") # 학교명에 '고등학교' 입력

# 각 시도교육청 선택하여 데이터 검색 후 csv 파일 다운받기
for office in tqdm_notebook(office_names):
    element = driver.find_element_by_id("sheet-filter-ATPT_OFCDC_SC_CODE")
    element.send_keys(office)
        
    driver.find_element_by_xpath("""//*[@id="sheet-search-button"]""").click()
    
    time.sleep(3)

    driver.find_element_by_xpath("""//*[@id="sheet-csv-button"]""").click()
    
    alert.accept()
    
    time.sleep(5)
    
driver.close()

#각 시도교욱청 17개의 csv파일 하나로 합치기
import pandas as pd
from glob import glob

school_diet_files = glob('C:/Users/dudtj/Desktop/데이터마이닝분석/급식식단 데이터/급식식단정보*csv')

tmp_raw = []                   ## 엑셀 파일에서 값 읽어서 tmp_raw에 저장

for file_name in school_diet_files:
    tmp = pd.read_csv(file_name, header=0)
    tmp_raw.append(tmp)
    
school_diet_raw = pd.concat(tmp_raw)   ##  pandas DataFrame으로 모든 엑셀 파일의 정보를 묶음

school_diet_raw.to_csv("급식식단 데이터.csv", encoding = 'utf-8-sig')